In [ ]:
import os
from pathlib import Path
import pandas as pd
import pickle
import numpy as np

df = pd.read_csv('data/selected_triplets.csv').dropna()
os.listdir('../models/11_03')
X_encoded_dict = np.load('../models/11_03/x_encoded_dict.pkl', allow_pickle=True)
X_encoded_dict = {os.path.split(k)[-1][:4]: v for k, v in X_encoded_dict.items()}

In [ ]:
X_encoded_dict['0015'].shape

In [ ]:
df['file_id'] = df.video_file.map(lambda p: os.path.split(p)[-1][:4])


In [ ]:
def get_segment_enc(encodings, seg_string):
    start, end = seg_string[1:-1].split(',')
    start, end = int(start), int(end)
    
    start_idx, end_idx = start // 4 - 15, end // 4 - 15
    assert end_idx > start_idx, print(start_idx, end_idx)
    idx = (start_idx + end_idx) // 2
    if idx >= len(encodings):
        return None
#     return encodings[idx]
    return encodings[start_idx:end_idx].mean(axis=0)

In [ ]:
df.anchor.map(lambda o: get_segment_enc(X_encoded_dict['0015'], o))
for sample in ['anchor', 'sample1', 'sample2']:
    df[f'{sample}_enc'] = df[sample].map(lambda o: get_segment_enc(X_encoded_dict['0015'], o))
    

In [ ]:
df['d1'] = (df['anchor_enc'] - df['sample1_enc']).map(np.linalg.norm)
df['d2'] = (df['anchor_enc'] - df['sample2_enc']).map(np.linalg.norm)
df['d3'] = (df['sample1_enc'] - df['sample2_enc']).map(np.linalg.norm)
df['pred_selected'] = (df['d1'] < df['d2']).map(lambda b: '1' if b else '2')
df['pred_correct'] = (df.selected == df.pred_selected)

In [ ]:
df[df['selected'] != '0']['pred_correct'].mean()

In [ ]:
df

In [ ]:
df[df['selected'] != '0'][['selected', 'pred_selected']].iloc[:10]

In [ ]:
df.iloc[2], X_encoded_dict.keys()